# Introduction to Machine Learning with scikit-learn
### Rachel Rakov
Welcome!  In this workshop, we are going to learn how to go through the process of doing *machine learning* on a set of data.   To do so, we will download a *corpus* of text data to work with, extract *features* from this data, and do *supervised* machine learning to our data, using a mathmatical algorithm to train a *classifier* which will then classify previously unseen data into a set of predefined categories.


"Machine learning is a research field that sits at the intersections of statistics, artificial intelligence, and computer science.  It is also known as *predictive analystics* or *statistical learning*."

-- Andreas Mueller, "Introduction to Machine Learning with Python".

## Key terms
- *machine learning*: An application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed


- *corpus*: A large collection of data.  In our case, this will be text data (although a corpus can contain any type of data)


- *features*: Properties that describe data entities for machine learning


- *feature representation, feature vector*: A set of features


- *supervised machine learning*:  A machine learning task of learning a function that maps an input to an output based on example input-output pairs


- *unsupervised machine learning*: A machine learning task used to draw inferences from datasets consisting of input data without labeled responses (lacks input-output pairs; only has input data)


- *algorithm*: A process or set of rules to be followed in calculations (or other problem-solving operations), particularly by a computer


- *classification*: An machine learning task used to predict a class label, which is a choice from a predefined list of possibilities

Sources: Wikipedia, Andreas Mueller's "Introduction to Machine Learning with Python"


## Goals of this workshop
In this workshop, you will learn the following skills:
- How to use skills from the NLTK workshop to build features for a classification task
- How to build a text classification system that can predict whether sentences belong to one category ("news") or another ("romance")
- How to prepare data for machine learning using *pandas*, a package for Python that helps to organize your data
    - Looks similar to an Excel spreadsheet
- How to use the scikit-learn package for Python to perform machine learning on the data
- How to evaluate the results of the classifier, helping to decide whether the classifier is effective
- How to adjust paramaters of a classifier to improve accuracy

### What do you need for this workshop?
- Python 3
    - You can also download the Jupyter Notebook for this lesson to follow along
- The Natural Lanugage Toolkit
    - We will be using both corpora and tools from this package
- pandas 
    - We will use this for data processing
    - Comes with Anaconda
- matplotlib
    - We will use this for visualizing our data
    - Comes with Anaconda
- sckiit-learn
    - We will use this for machine learning
    - Comes with Anaconda 

### Let's get started by importing some packages we will need for this workshop!
- The Brown Corpus: A text corpus of American English, split into fifteen different categories
- Part of speech taggers (POS): prebuilt functions that are designed to determine the part of speech of every word in the sentence you give them
- Pandas as pd: importing the Pandas toolkit and renaming it pd, so we don't have to type too much
- matplotlib.pyplot as plt: importing plotting tools from matplotlib and renaming them plt
    - ~~~
    %matplotlib inline
    ~~~
    We use the above code to ensure our images display clearly in the Jupyter notebook.

- sklearn: the scikit-learn machine learning toolkit



In [ ]:
import nltk
from nltk.corpus import brown
from nltk import pos_tag_sents
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

## What is classification?  Let's show an example using fruit!

### How would you describe apples to a computer?  How would they differ from oranges?
Remember, computers can only really understand numbers, true false values, and strings within a predefined set

![fruit3](images/fruit3.png)
Source: Andrew Rosenberg


Our fruit test shows us everything we need to do a classification machine learning test. For each item with a *label* (apple, orange, lemon), we use a series of values to try to capture machine-understandable information about the item.  These values are a *feature representation* of the item in question.  The features themselves, as we can see above, can be numeric, true/false values, or a string in a set of predefined strings.





### What if we had a new, unknown fruit?
![fruit2](images/fruit2.png)
Source: Andrew Rosenberg


Our fruit test is an example of a *classification* task.  Classification allows you to predict a *categorical* value.  This is a type of *supervised* machine learning, meaning we know the labels ahead of time and can give them to the machine learning algorithm so that it can be trained to knows what the categories of our data are.  This way, when it comes time to give the previously algorithm previously unseen data, it knows which categories it's looking for.

## Add a thing about people who do work like this (Hannah?)
- Viral Text (http://viraltexts.org/) - uses cluster analysis to find shared text propgated across networks
- Hip Hop Vocabulary: https://pudding.cool/2017/02/vocabulary/ - inverse frequency project
- Musical Genre Blurring in `Hamilton`: http://graphics.wsj.com/hamilton/ - genre classification
- Mood Mining: http://mood-mining.herokuapp.com/ - mood classification/sentiment analysis
- 

### Let's get to coding!
In this workshop we are going to *classify* two different sets of sentences from very different source material in the Brown corpus; one set of sentences from a corpus of news text, and the other set of sentences from a corpus of romance novel text. 

In [ ]:
from nltk.corpus import brown

#### For a list of catorigies in the Brown corpus, use the following code

In [ ]:
for cat in brown.categories():
    print (cat)

#### Get the sentences from each corpus

In [ ]:
news_sent = brown.sents(categories=["news"])
romance_sent = brown.sents(categories=["romance"])

### Practice 1:  Look at the first 5 sentences of each corpus
- What do you notice about the format of the data?

In [ ]:
print (news_sent[:5])
print ()
print (romance_sent[:5])


In [ ]:
print (news_sent[:5])
print ()
print (romance_sent[:5])


### What do you notice about the format of the data above?
Each sentence is already *tokenized* - split into a series of word and punctuation stringes, with whitespace removed. This saves us the time of having to do all of this work ourselves!

# Using data structures to manage data

To start to organize our data, let's put these sentences into a pandas *DataFrame*, an object which has a format very similar to an Excel spreadsheet.  We will first make two spreadsheets (one for news, and one for romance), and then combine them into one.  We will also add the category each sentences came from, which will be our *labels* for each sentence and its associated feature representation (which we will build ourselves).

In [ ]:
ndf = pd.DataFrame({'sentence': news_sent,
                    'label':'news'})
rdf = pd.DataFrame({'sentence':romance_sent, 
                    'label':'romance'})

In [ ]:
# combining two spreadsheets into 1
df = pd.concat([ndf, rdf])

Let's see what this DataFrame looks like!

In [ ]:
df 

In [ ]:
df.head()

### So how many texts do we have of each type?

In [ ]:
df['label'].value_counts()

### What if we want to visualize that information?
We first create a `figure` and `axes` on which to draw our charts using `plt.subplots()`. Each chart is one axes, and a figure can contain multiple charts. Our data is encapsulated in `df['label'].value_counts()`, which is itself a dataframe. We then tell the Pandas to visualize the dataframe as a bar chart using `.plot.bar(ax=ax, rot=0)`. The `ax` keyword tells Pandas which chart in the figure to plot, and the `rot` keyword controls the rotation of the x axis labels. 

In [ ]:
fig, ax = plt.subplots()
_ = df['label'].value_counts().plot.bar(ax=ax, rot=0)

We have slightly more news texts than romance texts, which we should keep in mind as we go ahead with classification.

### What should we use as features for our data set?  What did we use as features for our fruit example before?
![fruit3](images/fruit3.png)

### Now that we are using sentences, how can we best repersent each sentence as a series of values?

One idea is to use how many particular *parts of speech* the sentence contains.

- Nouns: Most basically described as a person, place, or thing.  Counting nouns can help determine how many topics are being discussed in a sentence.
- Adjectives: Descriptors of nouns (eg. "yellow", "angry", "charming").  Counting adjectives can help determine how often descriptive words are being added to nouns, which can demonstrate writing style.
- Adverbs: Descriptors of verbs (eg. "quickly", "hungrily", "annoyingly").  Counting adverbs can help determine how often the manner of the verb is modified, which can also demonstrate writing style.

#### Why might we want to use these parts of speech to distinguish between news sentences and romance sentences?


We will now compute all of the parts of speech on each sentence (row) in our dataframe.

### Practice 2:  Get the part of speech tags for each sentence in the dataframe
- Hint: 
~~~
df['sentence']
~~~

In [ ]:
# compute parts of speech on each sentence (row)
pos_all = pos_tag_sents(df['sentence'])
print (pos_all[:5])

### What's with those part of speech labels?  They aren't helpful at all!
The Penn Tagset, which NLTK uses for it's part of speech tagger, is not particularly intuitive.  Fortunately, they provide code that allows you to check what different tags stand for.

In [ ]:
# troubleshooting: https://github.com/nltk/nltk/issues/919
nltk.help.upenn_tagset("NN")
nltk.help.upenn_tagset("JJ")
nltk.help.upenn_tagset("RB")

**Troubleshooting**

If you get an error that says something like:
```
Resource 'help/tagsets/PY3/upenn_tagset.pickle' not found.
Please use the NLTK Downloader to obtain the resource: 
>>> nltk.download('tagsets')
Searched in:
- '/Users/user/nltk_data'
- '/usr/share/nltk_data'
- '/usr/local/share/nltk_data'
- '/usr/lib/nltk_data'
- '/usr/local/lib/nltk_data'
- ''
```
You may need to manually unzip the `tagsets.zip` file found in `nltk_data/help/`

### Write a function that calculates our features for us 
#### (In this case, numbers of nouns, adjectives, and adverbs that appear in the sentence)

Now we know the tags for the different parts of speech we want to count in each sentence.  Let's now write a function that will count the parts of speech to us, when given a part of speech tagged sentence (such as we have already in our DataFrame) and the part of speech we want to count (for example, "NN" to count the number of nouns in the sentence).

In [ ]:
def countPOS(pos_tag_sent, POS):
    pos_count = 0
    all_pos_counts = []
    for sentence in pos_tag_sent:
        for word in sentence:
            tag = word[1]
            if tag [:2] == POS:  
                pos_count = pos_count+1
        all_pos_counts.append(pos_count)
        pos_count = 0
    return all_pos_counts

We will now call this function three different times, one for each of the parts of speech we are counting.  As we finish counting them, we put the results into the DataFrame, saving us the trouble of having to do so later.

In [ ]:
df['NN'] = countPOS(pos_all, 'NN')
df['JJ'] = countPOS(pos_all, "JJ")
df['RB'] = countPOS(pos_all, "RB")

In [ ]:
df.head()

In [ ]:
df.tail()

### So how many POS types do we have for each type of text?
We can use the Pandas groupby function to aggregate our data based on unique values in any column of the data. Here we seperate our data into groups by the label type (news or romance) and then add together each texts count of nouns, adjectives, and adverbs. 

In [ ]:
df.groupby('label').sum()

### Let's visualize this data!
What do you notice about the data?  Do you think our features will be good at predicting news and romance sentences?  Which features do you think will be the most useful?

We can rotate the table using `.T` (transpose), which also changes the grouping of the data being plotted. 

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,  figsize=(10,5))
_ = df.groupby('label').sum().plot.bar(ax=ax1, rot=0)
_ = df.groupby('label').sum().T.plot.bar(ax=ax2, color=['gray','hotpink'], rot=0)



### Practice 3:  Save the dataframe to your computer as a csv file (comma separated value)

- Hint:
~~~
.to_csv()
~~~


In [ ]:
df.to_csv("df_news_romance.csv", index=False)